In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import lib
import plotting

In [4]:
from multiprocessing import Pool
import pickle
from collections import defaultdict

In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabaz_score
from sklearn.decomposition import PCA

In [6]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [7]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [8]:
sequence_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_sequences.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p")
results_prefix = "[WIKIDATA]"

In [9]:
df = pd.read_pickle(sequence_file)
labels = pd.read_pickle(labels_file)['label'].values
df.head()

,length,sequence,user_id
0,1511,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...",1
1,9,"[ENTITY_CREATE, BREAK, SITELINK_ADD, BREAK, ME...",1000036
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078
3,9,"[ENTITY_CREATE, BREAK, SITELINK_REMOVE, SITELI...",100008
4,44,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",100012


In [10]:
def calc_dist(sequence):
    dist, pivot = lib.calc_distribution(labels, sequence)
    return np.array(dist)

with Pool(cfg.getint("core", "num_cores")) as processor_pool:
    df['stat_dist'] = pd.Series(processor_pool.imap(calc_dist, tqdm(df['sequence'])))

Process ForkPoolWorker-20:
Process ForkPoolWorker-18:
Process ForkPoolWorker-12:
Process ForkPoolWorker-15:
Process ForkPoolWorker-17:
Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-14:
Process ForkPoolWorker-5:
Process ForkPoolWorker-13:
Process ForkPoolWorker-3:
Process ForkPoolWorker-11:
Process ForkPoolWorker-19:
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
Process ForkPoolWorker-16:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1:
Process ForkPoolWorker-10:
  File "/home/pkasper/python/Python-3.7.1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pkasper/python/Python-3.7.1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
kernel_range = np.arange(cfg.getint("kmeans", "clusters_min"), cfg.getint("kmeans", "clusters_max") + 1)
kernel_range

In [ ]:
vectors = np.array([np.array(x) for x in df['stat_dist'].values]) # ensure we have the data in numpy format
pca = PCA(n_components=3)
plot_vectors = pca.fit_transform(vectors)
print(pca.explained_variance_ratio_.cumsum())

In [ ]:
centroids = {}    
centroids_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_centroids.p")
load_centroids = cfg.getboolean("kmeans", "load_centroids") and os.path.isfile(centroids_file)
for num_centroids in tqdm(kernel_range):    
    kmeans = None

    print("Num Centroids: {n}".format(n=num_centroids))
    if load_centroids:
        with open(centroids_file, "rb") as dump_file:
            centroids = pickle.load(dump_file)
        kmeans = KMeans(n_clusters=num_centroids, init=centroids[num_centroids])
    else:
        kmeans = KMeans(n_clusters=num_centroids)
        
    kmeans.fit_predict(vectors)
    
    lbl = kmeans.labels_
    centroids[num_centroids] = kmeans.cluster_centers_

    
    silhouette_avg = silhouette_score(vectors, lbl)
    print("SILHOUETTE", silhouette_avg)
    calinski_score = calinski_harabaz_score(vectors, lbl)
    print("CALINSKI", calinski_score)
    sample_silhouette_values = silhouette_samples(vectors, lbl)
    
    store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "kmeans[{n}]".format(n=num_centroids))
    cluster_lbl = "kmeans_{n}".format(n=num_centroids)
    df[cluster_lbl] = lbl
    plotting.k_means(plot_vectors, num_centroids, lbl, sample_silhouette_values, silhouette_avg, store_path)
    
if cfg.getboolean("kmeans", "store_centroids"):
    with open(centroids_file, "wb") as dump_file:
        pickle.dump(centroids, dump_file)

In [ ]:
df.to_pickle(os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p"))

In [ ]:
for num_centroids in kernel_range:
    member_count = defaultdict(int)
    cluster = defaultdict(list)
    cluster_lbl = "kmeans_{n}".format(n=num_centroids)
    
    for i, user in tqdm(df.iterrows(), total=len(df)):
        cluster_id = user[cluster_lbl]
        member_count[cluster_id] += 1
        action_df = pd.DataFrame({"from": user['sequence']})
        action_df["to"] = action_df["from"].shift(periods=-1)
        cluster[cluster_id].append(action_df)
         
    for cluster_index in cluster:
        cluster_df = pd.concat(cluster[cluster_index])

        transition_count_pivot = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to']), labels)
        unnormalized = transition_count_pivot.transpose().sum().transpose()
        pivot = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to'], normalize="index"), labels)
        
        store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "clusters[{n}_{i}]".format(n=num_centroids, i=cluster_index))
        plotting.transition_matrix(unnormalized, pivot, transition_count_pivot, cluster_index, store_path)
        
        """
        transition_count_pivot_reduced = lib.stretch_pivot(transition_count_pivot, filtered_states)
        unnormalized_reduced = transition_count_pivot_reduced.transpose().sum().transpose()
        pivot_reduced = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to'], normalize="index"), filtered_states)
        
        store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "clusters[{n}_{i}]reduced".format(n=num_centroids, i=cluster_index))
        plotting.transition_matrix(unnormalized, pivot, transition_count_pivot, cluster_index, store_path)
        """
        